# Importing libraries

In [83]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import operator
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from numpy import unique
from numpy import reshape
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Conv2D, Dense, Dropout, BatchNormalization, Flatten, MaxPooling1D
from tensorflow.keras.utils import to_categorical

# Importing data

In [84]:
path = r'C:\Users\thoma\OneDrive\Dokumente\data analytics\ML_Ach1\ClimateWins'

In [85]:
path

'C:\\Users\\thoma\\OneDrive\\Dokumente\\data analytics\\ML_Ach1\\ClimateWins'

In [86]:
df = pd.read_csv(os.path.join(path, 'Data', 'Original Data', 'Dataset-weather-prediction-dataset-processed.csv'))
Pleasant_weather = pd.read_pickle(os.path.join(path, 'Data', 'Prepared Data', 'Pleasant_weather.pkl'))

In [87]:
df.shape

(22950, 170)

In [88]:
Pleasant_weather.shape

(22950, 15)

In [89]:
# Dropping 'DATE' & 'MONTH' columns
Weather = df.drop(['DATE','MONTH'], axis=1)

In [90]:
Weather.shape

(22950, 168)

In [91]:
Pleasant_weather.head()

,BASEL_pleasant_weather,BELGRADE_pleasant_weather,BUDAPEST_pleasant_weather,DEBILT_pleasant_weather,DUSSELDORF_pleasant_weather,HEATHROW_pleasant_weather,KASSEL_pleasant_weather,LJUBLJANA_pleasant_weather,MAASTRICHT_pleasant_weather,MADRID_pleasant_weather,MUNCHENB_pleasant_weather,OSLO_pleasant_weather,SONNBLICK_pleasant_weather,STOCKHOLM_pleasant_weather,VALENTIA_pleasant_weather
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [92]:
# Matching weather stations between Weather & Pleasant_weather data sets
Pleasant_weather.columns

Index(['BASEL_pleasant_weather', 'BELGRADE_pleasant_weather',
       'BUDAPEST_pleasant_weather', 'DEBILT_pleasant_weather',
       'DUSSELDORF_pleasant_weather', 'HEATHROW_pleasant_weather',
       'KASSEL_pleasant_weather', 'LJUBLJANA_pleasant_weather',
       'MAASTRICHT_pleasant_weather', 'MADRID_pleasant_weather',
       'MUNCHENB_pleasant_weather', 'OSLO_pleasant_weather',
       'SONNBLICK_pleasant_weather', 'STOCKHOLM_pleasant_weather',
       'VALENTIA_pleasant_weather'],
      dtype='object')

In [93]:
Weather.filter(regex=r'_temp_mean$').columns

Index(['BASEL_temp_mean', 'BELGRADE_temp_mean', 'BUDAPEST_temp_mean',
       'DEBILT_temp_mean', 'DUSSELDORF_temp_mean', 'GDANSK_temp_mean',
       'HEATHROW_temp_mean', 'KASSEL_temp_mean', 'LJUBLJANA_temp_mean',
       'MAASTRICHT_temp_mean', 'MADRID_temp_mean', 'MUNCHENB_temp_mean',
       'OSLO_temp_mean', 'ROMA_temp_mean', 'SONNBLICK_temp_mean',
       'STOCKHOLM_temp_mean', 'TOURS_temp_mean', 'VALENTIA_temp_mean'],
      dtype='object')

In [94]:
# Dropping the 3 weather stations missing from answers
Weather_stations = Weather.drop(columns=Weather.filter(regex=r'^(TOURS|ROMA|GDANSK)').columns)

In [95]:
Weather_stations.shape

(22950, 147)

# Looking for observation columns with missing entries over multiple years

In [96]:
# Any missing values within the weather data set
if df.isnull().values.any():
    print('Yes')
else:
    print('No')

No


# -> No missing values (NaNs)

# Further looking for observation columns with missing entries in at least one year

In [97]:
# Datatype
df['DATE'].dtype

dtype('int64')

In [98]:
# Formatting 'DATE'
df['DATE'] = pd.to_datetime(df['DATE'], format='%Y%m%d')

In [99]:
df['DATE'].dtype

dtype('<M8[ns]')

In [100]:
# Adding 'YEAR' to the DataFrame
df['YEAR'] = df['DATE'].dt.year

In [101]:
df.columns

Index(['DATE', 'MONTH', 'BASEL_cloud_cover', 'BASEL_wind_speed',
       'BASEL_humidity', 'BASEL_pressure', 'BASEL_global_radiation',
       'BASEL_precipitation', 'BASEL_snow_depth', 'BASEL_sunshine',
       ...
       'VALENTIA_humidity', 'VALENTIA_pressure', 'VALENTIA_global_radiation',
       'VALENTIA_precipitation', 'VALENTIA_snow_depth', 'VALENTIA_sunshine',
       'VALENTIA_temp_mean', 'VALENTIA_temp_min', 'VALENTIA_temp_max', 'YEAR'],
      dtype='object', length=171)

In [102]:
df['YEAR'].head()

0    1960
1    1960
2    1960
3    1960
4    1960
Name: YEAR, dtype: int32

In [103]:
# Number (counts) of missing values per year & for each obs column
missing_by_year = df.set_index('YEAR').isna().groupby('YEAR').sum()

In [104]:
missing_by_year

,DATE,MONTH,BASEL_cloud_cover,BASEL_wind_speed,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_snow_depth,BASEL_sunshine,...,VALENTIA_cloud_cover,VALENTIA_humidity,VALENTIA_pressure,VALENTIA_global_radiation,VALENTIA_precipitation,VALENTIA_snow_depth,VALENTIA_sunshine,VALENTIA_temp_mean,VALENTIA_temp_min,VALENTIA_temp_max
YEAR,,,,,,,,,,,,,,,,,,,,,
1960,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1961,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1962,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1963,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1964,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2019,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [105]:
# Filtering for observation columns with at least one missing entry in at least one year
Missing_filtered = missing_by_year.loc[:, missing_by_year.gt(0).any()]

In [106]:
Missing_filtered

""
YEAR
1960
1961
1962
1963
1964
...
2018
2019
2020


# -> No observation columns with missing entries

# Further EDA and search for any missing data entries

In [107]:
Weather2 = df.drop(columns=df.filter(regex=r'^(TOURS|ROMA|GDANSK)').columns)

In [108]:
Weather2.columns

Index(['DATE', 'MONTH', 'BASEL_cloud_cover', 'BASEL_wind_speed',
       'BASEL_humidity', 'BASEL_pressure', 'BASEL_global_radiation',
       'BASEL_precipitation', 'BASEL_snow_depth', 'BASEL_sunshine',
       ...
       'VALENTIA_humidity', 'VALENTIA_pressure', 'VALENTIA_global_radiation',
       'VALENTIA_precipitation', 'VALENTIA_snow_depth', 'VALENTIA_sunshine',
       'VALENTIA_temp_mean', 'VALENTIA_temp_min', 'VALENTIA_temp_max', 'YEAR'],
      dtype='object', length=150)

In [109]:
# Total number (counts) of data entries per year & for each obs colums
years_per_col = (Weather2.groupby('YEAR').count())

In [110]:
years_per_col

,DATE,MONTH,BASEL_cloud_cover,BASEL_wind_speed,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_snow_depth,BASEL_sunshine,...,VALENTIA_cloud_cover,VALENTIA_humidity,VALENTIA_pressure,VALENTIA_global_radiation,VALENTIA_precipitation,VALENTIA_snow_depth,VALENTIA_sunshine,VALENTIA_temp_mean,VALENTIA_temp_min,VALENTIA_temp_max
YEAR,,,,,,,,,,,,,,,,,,,,,
1960,366,366,366,366,366,366,366,366,366,366,...,366,366,366,366,366,366,366,366,366,366
1961,365,365,365,365,365,365,365,365,365,365,...,365,365,365,365,365,365,365,365,365,365
1962,365,365,365,365,365,365,365,365,365,365,...,365,365,365,365,365,365,365,365,365,365
1963,365,365,365,365,365,365,365,365,365,365,...,365,365,365,365,365,365,365,365,365,365
1964,366,366,366,366,366,366,366,366,366,366,...,366,366,366,366,366,366,366,366,366,366
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018,365,365,365,365,365,365,365,365,365,365,...,365,365,365,365,365,365,365,365,365,365
2019,365,365,365,365,365,365,365,365,365,365,...,365,365,365,365,365,365,365,365,365,365
2020,366,366,366,366,366,366,366,366,366,366,...,366,366,366,366,366,366,366,366,366,366


In [111]:
# Checking for date range boundaries
Weather2['DATE'].min(), Weather2['DATE'].max()

(Timestamp('1960-01-01 00:00:00'), Timestamp('2022-10-31 00:00:00'))

In [112]:
# Checking for years with(out) at least one data entry for each obs column
year_present = years_per_col > 0

In [113]:
year_present

,DATE,MONTH,BASEL_cloud_cover,BASEL_wind_speed,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_snow_depth,BASEL_sunshine,...,VALENTIA_cloud_cover,VALENTIA_humidity,VALENTIA_pressure,VALENTIA_global_radiation,VALENTIA_precipitation,VALENTIA_snow_depth,VALENTIA_sunshine,VALENTIA_temp_mean,VALENTIA_temp_min,VALENTIA_temp_max
YEAR,,,,,,,,,,,,,,,,,,,,,
1960,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
1961,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
1962,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
1963,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
1964,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
2019,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
2020,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True


In [114]:
# Dropping the last year 2022 - only full years
counts = Weather2.groupby('YEAR').count()

counts_ex2022 = counts.drop(2022, errors='ignore')

In [115]:
counts

,DATE,MONTH,BASEL_cloud_cover,BASEL_wind_speed,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_snow_depth,BASEL_sunshine,...,VALENTIA_cloud_cover,VALENTIA_humidity,VALENTIA_pressure,VALENTIA_global_radiation,VALENTIA_precipitation,VALENTIA_snow_depth,VALENTIA_sunshine,VALENTIA_temp_mean,VALENTIA_temp_min,VALENTIA_temp_max
YEAR,,,,,,,,,,,,,,,,,,,,,
1960,366,366,366,366,366,366,366,366,366,366,...,366,366,366,366,366,366,366,366,366,366
1961,365,365,365,365,365,365,365,365,365,365,...,365,365,365,365,365,365,365,365,365,365
1962,365,365,365,365,365,365,365,365,365,365,...,365,365,365,365,365,365,365,365,365,365
1963,365,365,365,365,365,365,365,365,365,365,...,365,365,365,365,365,365,365,365,365,365
1964,366,366,366,366,366,366,366,366,366,366,...,366,366,366,366,366,366,366,366,366,366
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018,365,365,365,365,365,365,365,365,365,365,...,365,365,365,365,365,365,365,365,365,365
2019,365,365,365,365,365,365,365,365,365,365,...,365,365,365,365,365,365,365,365,365,365
2020,366,366,366,366,366,366,366,366,366,366,...,366,366,366,366,366,366,366,366,366,366


In [116]:
counts_ex2022

,DATE,MONTH,BASEL_cloud_cover,BASEL_wind_speed,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_snow_depth,BASEL_sunshine,...,VALENTIA_cloud_cover,VALENTIA_humidity,VALENTIA_pressure,VALENTIA_global_radiation,VALENTIA_precipitation,VALENTIA_snow_depth,VALENTIA_sunshine,VALENTIA_temp_mean,VALENTIA_temp_min,VALENTIA_temp_max
YEAR,,,,,,,,,,,,,,,,,,,,,
1960,366,366,366,366,366,366,366,366,366,366,...,366,366,366,366,366,366,366,366,366,366
1961,365,365,365,365,365,365,365,365,365,365,...,365,365,365,365,365,365,365,365,365,365
1962,365,365,365,365,365,365,365,365,365,365,...,365,365,365,365,365,365,365,365,365,365
1963,365,365,365,365,365,365,365,365,365,365,...,365,365,365,365,365,365,365,365,365,365
1964,366,366,366,366,366,366,366,366,366,366,...,366,366,366,366,366,366,366,366,366,366
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017,365,365,365,365,365,365,365,365,365,365,...,365,365,365,365,365,365,365,365,365,365
2018,365,365,365,365,365,365,365,365,365,365,...,365,365,365,365,365,365,365,365,365,365
2019,365,365,365,365,365,365,365,365,365,365,...,365,365,365,365,365,365,365,365,365,365


In [117]:
# Checking for years with less than 365 entries & for each obs column
missing_data = counts_ex2022 < 365

In [118]:
missing_data

,DATE,MONTH,BASEL_cloud_cover,BASEL_wind_speed,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_snow_depth,BASEL_sunshine,...,VALENTIA_cloud_cover,VALENTIA_humidity,VALENTIA_pressure,VALENTIA_global_radiation,VALENTIA_precipitation,VALENTIA_snow_depth,VALENTIA_sunshine,VALENTIA_temp_mean,VALENTIA_temp_min,VALENTIA_temp_max
YEAR,,,,,,,,,,,,,,,,,,,,,
1960,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1961,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1962,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1963,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1964,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2018,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2019,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [119]:
# Checking for obs columns with at least one year of less than 365 entries
cols_missing_data = missing_data.any()

In [120]:
cols_missing_data

DATE                   False
MONTH                  False
BASEL_cloud_cover      False
BASEL_wind_speed       False
BASEL_humidity         False
                       ...  
VALENTIA_snow_depth    False
VALENTIA_sunshine      False
VALENTIA_temp_mean     False
VALENTIA_temp_min      False
VALENTIA_temp_max      False
Length: 149, dtype: bool

In [121]:
# Checking if there's any year with less than 365 data entries throughout the entire dataset
missing_data.any().any()

False

In [122]:
# Checking for the number of years with at least one data entry (date)
num_years_per_col = year_present.sum()

In [123]:
num_years_per_col

DATE                   63
MONTH                  63
BASEL_cloud_cover      63
BASEL_wind_speed       63
BASEL_humidity         63
                       ..
VALENTIA_snow_depth    63
VALENTIA_sunshine      63
VALENTIA_temp_mean     63
VALENTIA_temp_min      63
VALENTIA_temp_max      63
Length: 149, dtype: int64

In [124]:
# Checking again if there's at least one data entry per year & obs column
df2 = Weather2.copy()

yearly_present = (df2.groupby('YEAR').count()> 0)

In [125]:
yearly_present

,DATE,MONTH,BASEL_cloud_cover,BASEL_wind_speed,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_snow_depth,BASEL_sunshine,...,VALENTIA_cloud_cover,VALENTIA_humidity,VALENTIA_pressure,VALENTIA_global_radiation,VALENTIA_precipitation,VALENTIA_snow_depth,VALENTIA_sunshine,VALENTIA_temp_mean,VALENTIA_temp_min,VALENTIA_temp_max
YEAR,,,,,,,,,,,,,,,,,,,,,
1960,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
1961,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
1962,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
1963,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
1964,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
2019,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
2020,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True


# Looking for observation type columns with missing entries in at least one year

In [126]:
# Creating an observation type object
Observ_type = (pd.Series(df2.columns, index=df2.columns).str.split('_', n=1).str[1])

In [127]:
Observ_type

DATE                          NaN
MONTH                         NaN
BASEL_cloud_cover     cloud_cover
BASEL_wind_speed       wind_speed
BASEL_humidity           humidity
                         ...     
VALENTIA_sunshine        sunshine
VALENTIA_temp_mean      temp_mean
VALENTIA_temp_min        temp_min
VALENTIA_temp_max        temp_max
YEAR                          NaN
Length: 150, dtype: object

In [128]:
# Dropping 'DATE', 'YEAR' & 'MONTH' columns returning an index
Station_obs = df2.columns.difference(['DATE', 'MONTH', 'YEAR'])

In [129]:
type(Station_obs)

pandas.core.indexes.base.Index

In [130]:
# Missing (boolean) values over station–observation columns and original rows (dates)
missing_entries = df2[Station_obs].isna()

In [131]:
missing_entries

,BASEL_cloud_cover,BASEL_global_radiation,BASEL_humidity,BASEL_precipitation,BASEL_pressure,BASEL_snow_depth,BASEL_sunshine,BASEL_temp_max,BASEL_temp_mean,BASEL_temp_min,...,VALENTIA_cloud_cover,VALENTIA_global_radiation,VALENTIA_humidity,VALENTIA_precipitation,VALENTIA_pressure,VALENTIA_snow_depth,VALENTIA_sunshine,VALENTIA_temp_max,VALENTIA_temp_mean,VALENTIA_temp_min
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22945,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
22946,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
22947,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
22948,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [132]:
# Grouping by year with number (counts) of missing values per year
missing_by_year = missing_entries.groupby(df2['YEAR']).sum()

In [133]:
missing_by_year

,BASEL_cloud_cover,BASEL_global_radiation,BASEL_humidity,BASEL_precipitation,BASEL_pressure,BASEL_snow_depth,BASEL_sunshine,BASEL_temp_max,BASEL_temp_mean,BASEL_temp_min,...,VALENTIA_cloud_cover,VALENTIA_global_radiation,VALENTIA_humidity,VALENTIA_precipitation,VALENTIA_pressure,VALENTIA_snow_depth,VALENTIA_sunshine,VALENTIA_temp_max,VALENTIA_temp_mean,VALENTIA_temp_min
YEAR,,,,,,,,,,,,,,,,,,,,,
1960,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1961,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1962,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1963,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1964,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2019,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [134]:
# Missing entries by year and over observation type columns & aggregated across all stations
Missing_by_observ_year = (missing_by_year.T.groupby(Observ_type.loc[missing_by_year.columns]).sum().T)

In [135]:
Missing_by_observ_year.head()

,cloud_cover,global_radiation,humidity,precipitation,pressure,snow_depth,sunshine,temp_max,temp_mean,temp_min,wind_speed
YEAR,,,,,,,,,,,
1960,0,0,0,0,0,0,0,0,0,0,0
1961,0,0,0,0,0,0,0,0,0,0,0
1962,0,0,0,0,0,0,0,0,0,0,0
1963,0,0,0,0,0,0,0,0,0,0,0
1964,0,0,0,0,0,0,0,0,0,0,0


In [136]:
# Number of years with missing entries per observation type (aggregated across all stations)
years_missing_per_obs = (Missing_by_observ_year > 0).sum()

In [137]:
years_missing_per_obs

cloud_cover         0
global_radiation    0
humidity            0
precipitation       0
pressure            0
snow_depth          0
sunshine            0
temp_max            0
temp_mean           0
temp_min            0
wind_speed          0
dtype: int64

# -> In the current dataset, there are no observation type columns with data entries completely missing for a year, as well as no gaps of missing data entries and, therefore, no missing values at all within the entire data set
# -> I'd suggest speaking to management regarding the two observationtypes missing over multiple years, as this data set seems complete;        
#          - I'd recommend to ask management/stakeholders if the right data set has been handed over
#          - if I've picked the right data set for this task
#          - and further, I would ask if there's been any updates on the data, which haven't been communicated yet

In [138]:
# Shape
Weather2.shape

(22950, 150)

In [139]:
Weather2.columns

Index(['DATE', 'MONTH', 'BASEL_cloud_cover', 'BASEL_wind_speed',
       'BASEL_humidity', 'BASEL_pressure', 'BASEL_global_radiation',
       'BASEL_precipitation', 'BASEL_snow_depth', 'BASEL_sunshine',
       ...
       'VALENTIA_humidity', 'VALENTIA_pressure', 'VALENTIA_global_radiation',
       'VALENTIA_precipitation', 'VALENTIA_snow_depth', 'VALENTIA_sunshine',
       'VALENTIA_temp_mean', 'VALENTIA_temp_min', 'VALENTIA_temp_max', 'YEAR'],
      dtype='object', length=150)

In [140]:
# Exporting
Weather2.to_csv(os.path.join(path, 'Data', 'Prepared Data', 'Weather_NOTclean.csv'))

# Checking station-observation columns for completeness in Excel revealed that observation types 'wind_speed' & 'snow_depth' are missing across many of the stations

In [141]:
# Observation type counts
obs_type_counts = (Weather2.columns.str.split('_', n=1).str[1].value_counts().sort_values())

In [142]:
obs_type_counts

snow_depth           6
wind_speed           9
cloud_cover         14
humidity            14
pressure            14
global_radiation    15
precipitation       15
sunshine            15
temp_mean           15
temp_min            15
temp_max            15
Name: count, dtype: int64

# -> 2 observation types 'snow_depth' & 'wind_speed' are missing aross multiple weather stations and, hence, are being removed

# Filling the additional missing observation columns by copying from nearby stations

In [143]:
# Dropping 'DATE', 'MONTH' & 'YEAR' columns
Weather2 = Weather2.drop(columns=['DATE', 'MONTH', 'YEAR'], errors='ignore')

print(Weather2.columns.tolist())

['BASEL_cloud_cover', 'BASEL_wind_speed', 'BASEL_humidity', 'BASEL_pressure', 'BASEL_global_radiation', 'BASEL_precipitation', 'BASEL_snow_depth', 'BASEL_sunshine', 'BASEL_temp_mean', 'BASEL_temp_min', 'BASEL_temp_max', 'BELGRADE_cloud_cover', 'BELGRADE_humidity', 'BELGRADE_pressure', 'BELGRADE_global_radiation', 'BELGRADE_precipitation', 'BELGRADE_sunshine', 'BELGRADE_temp_mean', 'BELGRADE_temp_min', 'BELGRADE_temp_max', 'BUDAPEST_cloud_cover', 'BUDAPEST_humidity', 'BUDAPEST_pressure', 'BUDAPEST_global_radiation', 'BUDAPEST_precipitation', 'BUDAPEST_sunshine', 'BUDAPEST_temp_mean', 'BUDAPEST_temp_min', 'BUDAPEST_temp_max', 'DEBILT_cloud_cover', 'DEBILT_wind_speed', 'DEBILT_humidity', 'DEBILT_pressure', 'DEBILT_global_radiation', 'DEBILT_precipitation', 'DEBILT_sunshine', 'DEBILT_temp_mean', 'DEBILT_temp_min', 'DEBILT_temp_max', 'DUSSELDORF_cloud_cover', 'DUSSELDORF_wind_speed', 'DUSSELDORF_humidity', 'DUSSELDORF_pressure', 'DUSSELDORF_global_radiation', 'DUSSELDORF_precipitation', 'DU

In [144]:
# Stations & observation types
# Creating an index
station_cols = Weather2.columns

In [145]:
# Creating a DataFrame for stations & obs types
col_info = (pd.Series(station_cols).str.split('_', n=1, expand=True).rename(columns={0: 'station', 1: 'obs'}))

In [146]:
col_info

,station,obs
0,BASEL,cloud_cover
1,BASEL,wind_speed
2,BASEL,humidity
3,BASEL,pressure
4,BASEL,global_radiation
...,...,...
142,VALENTIA,snow_depth
143,VALENTIA,sunshine
144,VALENTIA,temp_mean
145,VALENTIA,temp_min


In [147]:
# Stations
col_info['station'].unique()

array(['BASEL', 'BELGRADE', 'BUDAPEST', 'DEBILT', 'DUSSELDORF',
       'HEATHROW', 'KASSEL', 'LJUBLJANA', 'MAASTRICHT', 'MADRID',
       'MUNCHENB', 'OSLO', 'SONNBLICK', 'STOCKHOLM', 'VALENTIA'],
      dtype=object)

In [148]:
# Checking if non-station-observation columns exist
col_info[col_info['obs'].isna()]

,station,obs


In [149]:
# Column presence table
presence = (col_info.assign(present=True).pivot(index='station', columns='obs', values='present'))

In [150]:
presence

obs,cloud_cover,global_radiation,humidity,precipitation,pressure,snow_depth,sunshine,temp_max,temp_mean,temp_min,wind_speed
station,,,,,,,,,,,
BASEL,True,True,True,True,True,True,True,True,True,True,True
BELGRADE,True,True,True,True,True,NaN,True,True,True,True,NaN
BUDAPEST,True,True,True,True,True,NaN,True,True,True,True,NaN
DEBILT,True,True,True,True,True,NaN,True,True,True,True,True
DUSSELDORF,True,True,True,True,True,True,True,True,True,True,True
HEATHROW,True,True,True,True,True,True,True,True,True,True,NaN
KASSEL,NaN,True,True,True,True,NaN,True,True,True,True,True
LJUBLJANA,True,True,True,True,True,NaN,True,True,True,True,True
MAASTRICHT,True,True,True,True,True,NaN,True,True,True,True,True


In [151]:
# Filtered column presence table
missing_obs = presence[presence.isna().any(axis=1)]
missing_obs

obs,cloud_cover,global_radiation,humidity,precipitation,pressure,snow_depth,sunshine,temp_max,temp_mean,temp_min,wind_speed
station,,,,,,,,,,,
BELGRADE,True,True,True,True,True,NaN,True,True,True,True,NaN
BUDAPEST,True,True,True,True,True,NaN,True,True,True,True,NaN
DEBILT,True,True,True,True,True,NaN,True,True,True,True,True
HEATHROW,True,True,True,True,True,True,True,True,True,True,NaN
KASSEL,NaN,True,True,True,True,NaN,True,True,True,True,True
LJUBLJANA,True,True,True,True,True,NaN,True,True,True,True,True
MAASTRICHT,True,True,True,True,True,NaN,True,True,True,True,True
MADRID,True,True,True,True,True,NaN,True,True,True,True,True
MUNCHENB,True,True,True,True,NaN,True,True,True,True,True,NaN


In [152]:
# Filtering for the 3 missing stations
stations_of_interest = ['KASSEL', 'MUNCHENB', 'STOCKHOLM']

missing_obs2 = presence.loc[stations_of_interest].loc[lambda x: x.isna().any(axis=1)]

In [153]:
missing_obs2

obs,cloud_cover,global_radiation,humidity,precipitation,pressure,snow_depth,sunshine,temp_max,temp_mean,temp_min,wind_speed
station,,,,,,,,,,,
KASSEL,NaN,True,True,True,True,NaN,True,True,True,True,True
MUNCHENB,True,True,True,True,NaN,True,True,True,True,True,NaN
STOCKHOLM,True,True,NaN,True,True,NaN,True,True,True,True,NaN


In [154]:
# Filling in the missing values
Weather2['KASSEL_cloud_cover'] = Weather2['LJUBLJANA_cloud_cover']
Weather2['MUNCHENB_pressure'] = Weather2['SONNBLICK_pressure']
Weather2['STOCKHOLM_humidity'] = Weather2['OSLO_humidity']

In [155]:
# Observation type counts
Weather2.columns.str.split('_', n=1).str[1].value_counts()

cloud_cover         15
humidity            15
pressure            15
global_radiation    15
precipitation       15
sunshine            15
temp_mean           15
temp_min            15
temp_max            15
wind_speed           9
snow_depth           6
Name: count, dtype: int64

In [156]:
# Dropping the 2 widely missing observation types
Weather3 = Weather2.drop(columns=Weather2.filter(regex=r'_(wind_speed|snow_depth)$').columns)

In [157]:
# Observation type counts
Weather3.columns.str.split('_', n=1).str[1].value_counts()

cloud_cover         15
humidity            15
pressure            15
global_radiation    15
precipitation       15
sunshine            15
temp_mean           15
temp_min            15
temp_max            15
Name: count, dtype: int64

In [158]:
Weather3.columns

Index(['BASEL_cloud_cover', 'BASEL_humidity', 'BASEL_pressure',
       'BASEL_global_radiation', 'BASEL_precipitation', 'BASEL_sunshine',
       'BASEL_temp_mean', 'BASEL_temp_min', 'BASEL_temp_max',
       'BELGRADE_cloud_cover',
       ...
       'VALENTIA_pressure', 'VALENTIA_global_radiation',
       'VALENTIA_precipitation', 'VALENTIA_sunshine', 'VALENTIA_temp_mean',
       'VALENTIA_temp_min', 'VALENTIA_temp_max', 'KASSEL_cloud_cover',
       'MUNCHENB_pressure', 'STOCKHOLM_humidity'],
      dtype='object', length=135)

In [159]:
Weather3.shape

(22950, 135)

In [160]:
Pleasant_weather.shape

(22950, 15)

In [161]:
# Exporting
Weather3.to_csv(os.path.join(path, 'Data', 'Prepared Data', 'Weather_unsc_clean.csv'))